In [1]:
from transformers import pipeline

In [2]:
classificador = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

Device set to use cuda:0


In [3]:
categorias = ['eletronics', 'food', 'toys', 'books']

In [4]:
predicao = classificador("""
    Latest model of smartphone with 5G connection and 128GB internal storage
""", candidate_labels=categorias)

In [5]:
predicao

{'sequence': '\n    Latest model of smartphone with 5G connection and 128GB internal storage\n',
 'labels': ['eletronics', 'books', 'toys', 'food'],
 'scores': [0.8386489152908325,
  0.05798647925257683,
  0.05383487418293953,
  0.04952971637248993]}

In [6]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')

classifier = ZeroShotClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    hypothesis_template='{}',
    encoder_decoder=True
)

You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0


In [7]:
descricao = """ 
    Este water gel leve e refrescante, proposiciona uma hidratação intensa e duradoura, ajuda a aliviar a sensação de pele seca e repuxada,
"""

In [8]:
categorias_candidatas = ['beleza', 'cozinha', 'livros']

In [9]:
resultado = classifier(descricao, categorias_candidatas, multi_label=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
resultado

{'sequence': ' \n    Este water gel leve e refrescante, proposiciona uma hidratação intensa e duradoura, ajuda a aliviar a sensação de pele seca e repuxada,\n',
 'labels': ['beleza', 'livros', 'cozinha'],
 'scores': [0.5260359644889832, 0.010608719661831856, 0.004840693436563015]}

In [11]:
import pandas as pd

resultado = pd.DataFrame(resultado).drop(['sequence'], axis=1)
resultado

,labels,scores
0,beleza,0.526036
1,livros,0.010609
2,cozinha,0.004841


### Aplicando modelo aos dados

In [12]:
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/hugging_face/main/Dados/descricoes_produtos.csv')
dados.head()

,Descrição
0,Liquidificador de alta potência com jarra de v...
1,"Forno Micro-ondas de 20 litros, com menu desco..."
2,Máquina de café espresso com reservatório de á...
3,Torradeira com capacidade para quatro fatias e...
4,"Panela elétrica multifuncional que cozinha, as..."


In [13]:
categorias = ['eletromesticos', 'eletronicos', 'beleza', 'brinquedos']

def categoriar(descricao):
    resultado  = classifier(descricao, categorias, multi_label=False)
    categoria_max = max(zip(resultado['labels'], resultado['scores']), key=lambda x: x[1])[0]
    return categoria_max

In [15]:
dados['Categoria'] = dados['Descrição'].apply(categoriar)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [16]:
dados

,Descrição,Categoria
0,Liquidificador de alta potência com jarra de v...,eletronicos
1,"Forno Micro-ondas de 20 litros, com menu desco...",eletronicos
2,Máquina de café espresso com reservatório de á...,eletronicos
3,Torradeira com capacidade para quatro fatias e...,eletronicos
4,"Panela elétrica multifuncional que cozinha, as...",eletromesticos
5,Smartphone com 128GB de armazenamento e câmera...,eletronicos
6,"Smartwatch monitoramento de passos, frequência...",eletronicos
7,Notebook gamer com placa de vídeo potente e te...,eletronicos
8,"Smartphone com tela super retina XDR de 6,7 po...",beleza
9,Câmera digital com zoom óptico de 20x e vídeos...,eletronicos
